# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [ ]:
import os
import glob
import psycopg2
import numpy as np
import pandas as pd
from sql_queries import *
from datetime import datetime

In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [ ]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [ ]:
song_files = get_files(filepath)

In [ ]:
filepath = 'data/song_data'

In [ ]:
df_song=df[['song_id', 'title', 'artist_id', 'year', 'duration']]

#get first row df_song
# debug: convert array to a song list
#song_data = df_song_1.values.tolist()

#w2 using this df.values: for matching dtype in dataframe and postgreSQL
df_song_1 = df_song.values[0]
song_data = df_song_1.tolist()

print(type(song_data))
print(song_data)

# print("\n to debug dtype for each in df_song song_id :{} \n title: {} \n artist_id: {} \n duration: {}".f(song_data[0]))
#print("to debug dtype for each in song_id :{} \n duration : {} \n year: {} \n title: {}".format(type(song_data[0]), type(song_data[4]), type(song_data[3]), type(song_data[1])))

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
df_song=df[['song_id', 'title', 'artist_id', 'year', 'duration']]

#get first row df_song
# debug: convert array to a song list
#song_data = df_song_1.values.tolist()

#w2 using this df.values: for matching dtype in dataframe and postgreSQL
df_song_1 = df_song.values[0]
song_data = df_song_1.tolist()

print(type(song_data))
print(song_data)

# print("\n to debug dtype for each in df_song song_id :{} \n title: {} \n artist_id: {} \n duration: {}".f(song_data[0]))
#print("to debug dtype for each in song_id :{} \n duration : {} \n year: {} \n title: {}".format(type(song_data[0]), type(song_data[4]), type(song_data[3]), type(song_data[1])))

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
df_artist=df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]

df_artist_1 = df_artist.values[0]
artist_data = df_artist_1.tolist()

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
filepath = 'data/log_data'

In [ ]:
log_files = get_files(filepath)

In [ ]:
df = pd.read_json(log_files[0], lines=True)

#to see 5rows of log_data files
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df =  pd.read_json(log_files[0], lines=True)

#filter Page is "NextSong"
df = df[df['page']=='NextSong']
df.head()

In [ ]:
rawTime = df[['ts']]
rawTime.head()
print()
print("timestamp prior to convert {} \n".format(rawTime.head()))

#first row ts and convert to datetime in pandas
ts_dtime_S = []

#count = 0
for index, row in rawTime.iterrows():
#    count += 1
    ts_dtime = pd.to_datetime(row["ts"])
    ts_dtime_S.append(pd.Series(ts_dtime))

#print("type of ts_dtime {} \n".format(type(ts_dtime)))
#print("to check hour year month {}  {} \n".format(ts_dtime_S[0].dt.month, ts_dtime_S[1].dt.month))

#debug
#print("number of ts_dtime_S \n {} \n".format(len(ts_dtime_S)))
#print("to check type of ts_dtime_S element : {} \n".format(type(ts_dtime_S[0])))
#print("to check type of ts_dtime_S element : {} \n".format(ts_dtime_S[0]))


In [ ]:
time_data = []
time_data_e = []

#c2=0
# convert to timestamp:  datetime.fromtimestamp(xxxx/1000)
for ele in ts_dtime_S:
    #dt64 = np.datetime64(ele.values[0])
    #ts = (dt64 - np.datetime64('1970-01-01T00:00:00.000000000'))/(np.timedelta64(1, 'ms'))
    
    
    time_data_e.append(np.datetime64(ele.values[0]).astype(datetime))

    time_data_e.append(ele.dt.hour.values[0])
    time_data_e.append(ele.dt.day.values[0])
    time_data_e.append(ele.dt.weekofyear.values[0])
    time_data_e.append(ele.dt.month.values[0])
    time_data_e.append(ele.dt.year.values[0])
    time_data_e.append(ele.dt.weekday.values[0])
    time_data.append(time_data_e)
    time_data_e = []

#to see time_data list
print(time_data_e)
print()
print(time_data)

column_labels = ['ts','hour','day','weekofyear','month','year','weekday']

In [ ]:
# for debug
#print((dict(zip(column_labels,time_data))))
#dict_time=dict(zip(column_labels,time_data))
#time_df = pd.DataFrame(dict_time,index=['row0'])
arr = np.array(time_data)
time_df = pd.DataFrame(arr,columns=column_labels)
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
# insert each time row in time table into the time_table
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
# userId, firstName, lastName, gender, level
df_user=df[['userId', 'firstName', 'lastName', 'gender', 'level']]

df_user_1 = df_user.values[0]
user_data = df_user_1.tolist()

#debug
#print(df_user_data)

user_column=['userId', 'firstName', 'lastName', 'gender', 'level']
user_dict=dict(zip(user_column,user_data))

user_df = pd.DataFrame(user_dict,index=['row0'])

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:

for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()

    #debug: to see result
    #print("results after SELECT: {}".format(results))

    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    # songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
    # ts (convert to dt)

    # userId, firstName, lastName, gender, level
    # extract from log data
    df_songplay_aux=df[['ts','userId','level','sessionId','location','userAgent']]

    # to get the first row and transform to a list
    df_songplay_aux_1 = df_songplay_aux.values[0]
    songplay_aux_data = df_songplay_aux_1.tolist()
    
    #convert ts to timestamp, applied above ts_dtime 
    # ts_dtime
    songplay_aux_data[0] = datetime.fromtimestamp(songplay_aux_data[0]/1000)

    #debug
    #print("to check songplay_aux_data {} \n".format(songplay_aux_data))

    #deal with None for songID and artistID
    if songid is None:
        songid = 0
    if artistid is None:
        artistid = 0

    #debug
    # to check songid and artistid
    #print("to check songid {}  and artistid \n".format(songid,artistid))

    # to append songplayid
    #songplay_aux_data.insert(index,0)  -< using SERIAL when CREAT TABLE
    # location of songid and artistid is not counted for songplay_id
    #songplay_aux_data.insert(3,songid)
    #songplay_aux_data.insert(4,artistid)
    
    songplay_aux_data.insert(0,index)
    songplay_aux_data.insert(4,songid)
    songplay_aux_data.insert(5,artistid)

    #debug
    #print("to check songplay_aux_data after insert 2 boys {} \n".format(songplay_aux_data))
    # assumn index is increment from df was read from whole file json

    #option1
    songplay_column = ['songplay_id','ts','userId','level','song_id','artist_id','sessionId','location','userAgent']

    # option2: songplay_id is primary key : use SERIAL auto increment by postgreSQL
    #songplay_column = ['ts','userId','level','song_id','artist_id','sessionId','location','userAgent']

    songplay_dict = dict(zip(songplay_column,songplay_aux_data))

    #debug
    #print("to see songplay_dcit = {} \n".format(songplay_dict))
    # songplay_df = pd.DataFrame(songplay_dict)

    # songplay_data = ()
    cur.execute(songplay_table_insert, songplay_aux_data)
    conn.commit()


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.